In [1]:
import pandas as pd
import os
from preProcess import generateData
from Classifiers import cnn, main, vectorize

In [2]:
os.chdir('../../')

In [3]:
os.getcwd()

'c:\\Users\\hannah.alexander\\OneDrive - Ascent Software Ltd\\Documents\\Internal projects\\disaster-tweet-hackathon\\DHG_team\\src'

In [10]:
#Read in full training data
train_data = pd.read_csv("data/train.csv")

In [40]:
#Read in test data
holdback_data = pd.read_csv("data/test.csv")

In [41]:
#Do preprocessing
train_preprocessed_text, train_preprocessed_labels= generateData("data/train.csv", dataset_type = "train")
holdback_data_preprocessed = generateData("data/test.csv", dataset_type = "test")

In [27]:
#Fit CNN on full training data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

X_train_text, X_val_text, y_train, y_val = train_test_split(
        train_preprocessed_text, train_preprocessed_labels, test_size=0.2, random_state=1000
    )

# fit vectorizer only on training data
vectorizer = vectorize(X_train_text)

# change data set from words to numeric values
#X_train = vectorizer.transform(X_train_text).toarray()
#X_test = vectorizer.transform(X_val_text).toarray()
encoder = LabelBinarizer()
encoder.fit(y_train)
Y_train = encoder.transform(y_train).ravel()
Y_test = encoder.transform(y_val).ravel()



In [34]:
type(y_val)

list

In [35]:
cnn_model = cnn(X_train_text, X_val_text, Y_train, Y_test)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 32, 100)           1366500   
                                                                 
 dropout_6 (Dropout)         (None, 32, 100)           0         
                                                                 
 conv1d_2 (Conv1D)           (None, 23, 128)           128128    
                                                                 
 global_max_pooling1d_2 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 10)                1290      
                                                      

INFO:tensorflow:Assets written to: outputs\Model_ID20221013170321_cnn\assets


INFO:tensorflow:Assets written to: outputs\Model_ID20221013170321_cnn\assets


INFO:tensorflow:Assets written to: outputs\Model_ID20221013170321_cnn\assets


INFO:tensorflow:Assets written to: outputs\Model_ID20221013170321_cnn\assets


Training Accuracy: 0.8969
Testing Accuracy:  0.8012
44/44 [==============================] - 0s 6ms/step
{'accuracy_score_test': 0.8011527377521613,
 'accuracy_score_train': 0.896936936936937,
 'confusion_matrix_test': array([[777,  67],
       [209, 335]], dtype=int64),
 'confusion_matrix_train': array([[3208,   55],
       [ 517, 1770]], dtype=int64),
 'f1_score_test': 0.7082452431289641,
 'f1_score_train': 0.8608949416342412,
 'model_id': '20221013170321_cnn',
 'precision_score_test': 0.8333333333333334,
 'precision_score_train': 0.9698630136986301,
 'recall_score_test': 0.6158088235294118,
 'recall_score_train': 0.7739396589418452}


In [36]:
#Read in saved model
import tensorflow as tf
model = tf.keras.models.load_model('outputs/Model_ID20221013152438_cnn')


In [17]:
model

In [12]:
model.save('outputs/cnn_model')

INFO:tensorflow:Assets written to: outputs/cnn_model\assets


INFO:tensorflow:Assets written to: outputs/cnn_model\assets


In [42]:
# Predict test data
# tokenizer fit on training data
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(holdback_data_preprocessed)

# blogs are turned to sequenuences by means of the tokenizer
holdback_data_preprocessed = tokenizer.texts_to_sequences(holdback_data_preprocessed)

holdback_data_preprocessed = pad_sequences(holdback_data_preprocessed, padding='post', maxlen=32)
results = model.predict(holdback_data_preprocessed)

# convert to class
threshold = 0.5
results = [1 if x >= threshold else 0 for x in results]

102/102 [==============================] - 1s 7ms/step


In [44]:
holdback_data["results"] = results

In [45]:
#save results in kaggle format
holdback_data

,id,keyword,location,text,results
0,0,NaN,NaN,Just happened a terrible car crash,1
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",0
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,1
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,0
...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,0
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,1
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,1
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,1
